In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from temp import *
from comUsed import formatXg

In [ ]:
clu_Index = json_load(r'D:\pycharmProject\logic\计算道路指标(通过sections)\speed&travelTime\clu_Index.json')
clu_time = {}
for cluId, index in clu_Index.items():
    clu_time[cluId] = index[1]
print(clu_time)
timeL = list(clu_time.values())
plt.plot(timeL)
plt.show()

cluId_MAX = []
for cluId, time in clu_time.items():
    if time > 1000:
        cluId_MAX.append(cluId)
time_ave = np.average(timeL)
print(cluId_MAX, time_ave)
for cluId in cluId_MAX:
    clu_Index[cluId][1] = time_ave
timeL = list(clu_time.values())
plt.plot(timeL)
plt.show()
# json_dump(clu_Index, r'D:\pycharmProject\logic\计算道路指标(通过sections)\speed&travelTime\clu_Index.json')

In [ ]:
clu_index = json_load(r'D:\pycharmProject\logic\计算道路指标(通过sections)\speed&travelTime\clu_Index.json')
clu_length = json_load(r'D:\pycharmProject\logic\计算道路指标(通过sections)\roadLength\cluId_length.json')
print(len(list(clu_index)), len(list(clu_length)))
dic = {}
staL = json_load(r'D:\pycharmProject\logic\dataSet\staIdL.json')
for sta in staL:
    dic[sta[2]] = sta[0:2]
json_dump(dic, 'dic_staId_lnglat.json')
# print(dic)

In [ ]:
fNameL, pathL = getPath(r'D:\pycharmProject\logic\3.线路网络\route2station')
dic = {}
for fName, path in zip(fNameL, pathL):
    stations = json_load(path)
    stations2 = []
    for station in stations:
        if station[2] > 10000:
            stations2.append(station)
    dic[fName.replace('.json', '')] = stations2
# print(dic)
json_dump(dic, r'D:\pycharmProject\busAnalyze-backend\data\road2stations.json')

In [ ]:
# 拼接线路
section2stations = json_load(r'D:\pycharmProject\logic\计算映射\output\section2stations.json')
dic = defaultdict(list)
routeNameL = []
# key 102_down_01
for routeName, stations in section2stations.items():
    routeName = routeName.split('_')[0] + '_' + routeName.split('_')[1]
    if routeName not in routeNameL:
        routeNameL.append(routeName)

for routeName in routeNameL:
    houZui = ''
    for i in range(1, 51):
        if i < 10:
            houZui = '_0'+str(i)
        else:
            houZui = '_'+str(i)
        sectionId = routeName+houZui
        if sectionId in list(section2stations.keys()):
            stations = section2stations[sectionId]
            dic[routeName].extend(stations)
dic2 = defaultdict(list)
for routeName, stations in dic.items():
    for i, station in enumerate(stations):
        if i != 0:
            if station!=stations[i-1]:
                dic2[routeName].append(station)

json_dump(dic2, 'route2stations.json')

In [ ]:
fNameL, pathL = getPath(r'D:\pycharmProject\logic\3.线路网络\route2station')
dic = {}
for name, path in zip(fNameL, pathL):
    name = name.replace('.json', '')
    dic[name] = json_load(path)
json_dump(dic, 'route2stations.json')

In [ ]:
import pandas as pd
from temp import *
import time
fNameL, pathL = getPath(r'D:\pycharmProject\logic\dataBase\section_driving\section_drivings')

for path in pathL:
    df = pd.read_csv(path)
    timeList = df['start_date_time'].values.tolist()
    timeList = [time for time in timeList if time != '-1' and time != 'start_date_time']
    timeList = [time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S')) for t in timeList]
    # print(timeList)
    break

tL = []
for i, t in enumerate(timeList):
    tL.append([i, t])
json_dump(tL, r'D:\pycharmProject\busAnalyze-backend\data\tL.json')

In [ ]:
from temp import *
import time

timeStamp = 1631600000
timeArray = time.localtime(timeStamp)
otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
print(otherStyleTime)

timeStamp = 1631100000
timeArray = time.localtime(timeStamp)
otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
print(otherStyleTime)

In [ ]:
# I want [时间范围:{sectionId: [{记录1, 记录2, 记录3}]}}]
import datetime
from temp import *
def timeSub(time1, time2):
    # a = datetime.strptime(str(time1), "%Y-%m-%d %H:%M:%S")
    # b = datetime.strptime(str(time2), "%Y-%m-%d %H:%M:%S")
    return datetime.datetime.timestamp(datetime.datetime.strptime(str(time1), '%Y-%m-%d %H:%M:%S')) - datetime.datetime.timestamp(datetime.datetime.strptime(str(time2), '%Y-%m-%d %H:%M:%S'))

# 生成一个dic_time_sectionId: path
def timeFilter(df, t1='2021-09-09 10:00:00', t2='2021-09-09 11:00:00'):
    # 筛选时间
    df = df[(df['start_date_time'] != '-1') & (df['start_date_time'] != 'start_date_time')]
    filter = df['start_date_time'].apply(lambda x: (timeSub(x, t1)) > 0 and (timeSub(t2, x)) > 0)
    df = df[filter]
    return df

def getJson_t_section(t1='2021-09-09 10:00:00', t2='2021-09-09 11:00:00'):
    # 输入时间,获得json块 时间_sectionId: [记录]
    dic = defaultdict(list)
    jsonList = []
    fNameL, pathL = getPath(r'D:\pycharmProject\logic\dataBase\section_driving\section_drivings')
    for fName, path in zip(fNameL, pathL):
        df = pd.read_csv(path)
        # 筛选 时间粒度 1小时
        df = timeFilter(df, t1, t2)
        valuesList = df.values.tolist()
        for i, value in enumerate(valuesList):
            valuesList[i][2] = json.loads(valuesList[i][2].replace("'", '"'))
        jsonList = [dict(zip(df.columns.tolist() ,value)) for value in valuesList]
    sectionNames = []
    for item in jsonList:
        if item['sectionId'] not in sectionNames:
            sectionNames.append(item['sectionId'])
        dic[item['sectionId']].append(item)
    json_dump(dic, 'temp.json')

getJson_t_section()

In [5]:
# section长度 dic_sectionId_length
from temp import *
dic_sectionId_length = {}
fNameL, pathL = getPath(r'D:\pycharmProject\logic\dataBase\section\sections')
for path, name in zip(pathL, fNameL):
    points = json_load(path)[4]
    length = getLength(points)
    dic_sectionId_length[name.replace('.json', '')] = length
print(dic_sectionId_length)
json_load('section2length.json')

{'101_down_00': 64.72594353079185, '101_down_01': 2715.7272739013624, '101_down_02': 1540.6971495112991, '101_down_03': 78.27280461296017, '101_down_04': 7239.975551430666, '101_down_05': 15697.728727206424, '101_down_06': 150.538594665934, '101_up_00': 97.65537369218659, '101_up_01': 874.7898420658227, '101_up_02': 476.98429761244785, '101_up_03': 3505.981817540916, '101_up_04': 1168.980102604215, '101_up_05': 304.8187160632805, '101_up_06': 612.854235714609, '101_up_07': 7767.848070274133, '101_up_08': 7555.5877477349595, '101_up_09': 69.00800746732074, '101_up_10': 1579.0002487163372, '101_up_11': 2153.425162805609, '101_up_12': 510.89213368381826, '101_up_13': 907.2598496639218, '103_down_00': 3784.3523687458987, '103_down_01': 551.2659911439163, '103_down_02': 425.01809668759125, '103_down_03': 1971.7989843668872, '103_down_04': 917.4528415596072, '103_down_05': 1023.1436123528956, '103_down_06': 647.6452180340123, '103_down_07': 4316.227009912566, '103_down_08': 119.8771877673695